In [8]:
!az extension add -n ml -y 

Extension 'ml' is already installed.


In [9]:
!az login

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RPBG885HA to authenticate.
The following tenants don't contain accessible subscriptions. Use 'az login --allow-no-subscriptions' to have tenant level access.
b1c14d5c-3625-45b3-a430-9552373a0c2f 'SAS'
[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "d6f18478-178f-450d-8f3e-a891b4d83dd0",
    "id": "59ea72cc-1f3e-4d29-bd60-242d81b57b05",
    "isDefault": false,
    "managedByTenants": [],
    "name": "Visual Studio Enterprise Subscription",
    "state": "Enabled",
    "tenantId": "d6f18478-178f-450d-8f3e-a891b4d83dd0",
    "user": {
      "name": "osomorog@microsoft.com",
      "type": "user"
    }
  },
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
    "id": "316a14b3-f94a-4267-a76a-497e589ef0a8",
    "isDefault": false,
    "managedByTenants": [],
    "name": "Ranker training",
    "state": "Enabled",
    "tenantId": "72f988bf-86f

In [11]:
import subprocess

#Get MLFLow UI through the Azure ML CLI v2 and convert to string
MLFLOW_TRACKING_URI = subprocess.run(["az", "ml", "workspace", "show", "--query", "mlflow_tracking_uri", "-o", "tsv"], stdout=subprocess.PIPE, text=True)
MLFLOW_TRACKING_URI = str(MLFLOW_TRACKING_URI.stdout).strip()

## Make sure the MLFLow URI looks something like this: 
## azureml://westus.api.azureml.ms/mlflow/v1.0/subscriptions/<Sub-ID>/resourceGroups/<RG>/providers/Microsoft.MachineLearningServices/workspaces/<WS>
print("MLFlow Tracking URI:", MLFLOW_TRACKING_URI)

MLFlow Tracking URI: azureml://westus.api.azureml.ms/mlflow/v1.0/subscriptions/95a911b6-47f7-4a8b-be9b-c1c2bf56579b/resourceGroups/osomorog/providers/Microsoft.MachineLearningServices/workspaces/mlflowworkspace


In [12]:
## Set the MLFLOW TRACKING URI
import mlflow
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [15]:
import mlflow
experiment_name = 'experiment_with_mlflow'
mlflow.set_experiment(experiment_name)

In [16]:
import os
from random import random

with mlflow.start_run() as mlflow_run:
    mlflow.log_param("hello_param", "world")
    mlflow.log_metric("hello_metric", random())
    os.system(f"echo 'hello world' > helloworld.txt")
    mlflow.log_artifact("helloworld.txt")


In [11]:
from mlflow.tracking import MlflowClient
# Use MlFlow to retrieve the run that was just completed
client = MlflowClient()
run_id = mlflow_run.info.run_id
finished_mlflow_run = MlflowClient().get_run(run_id)

metrics = finished_mlflow_run.data.metrics
tags = finished_mlflow_run.data.tags
params = finished_mlflow_run.data.params

print(metrics,tags,params)


{'hello_metric': 0.2773886086483984} {'mlflow.user': 'azureuser', 'mlflow.source.name': '/anaconda/envs/azureml_py38/lib/python3.8/site-packages/ipykernel_launcher.py', 'mlflow.source.type': 'LOCAL', 'mlflow.rootRunId': '09bb00f9-3d7f-4f6e-93b7-856c0c30c68b'} {'hello_param': 'world'}


In [12]:
client.list_artifacts(run_id)

[<FileInfo: file_size=-1, is_dir=False, path='helloworld.txt'>]

In [13]:
client.download_artifacts(run_id, "helloworld.txt", ".")

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/osomorog4/code/Users/osomorog/MLFlow_Scenarios/helloworld.txt'

In [14]:
### Compare and query:
from mlflow.entities import ViewType

all_experiments = [exp.experiment_id for exp in MlflowClient().list_experiments()]
query = "metrics.hello_metric > 0"
runs = mlflow.search_runs(experiment_ids=all_experiments, filter_string=query, run_view_type=ViewType.ALL)

runs.head(10)


,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.hello_metric,params.hello_param,tags.mlflow.rootRunId,tags.mlflow.user,tags.mlflow.source.name,tags.mlflow.source.type
0,9d6edb9b-f9d8-4870-bc6f-5d192c28e81a,18168c22-8581-4420-9b3b-4301464d25c3,FINISHED,,2021-10-13 00:17:53.632000+00:00,2021-10-13 00:21:00.184000+00:00,0.272088,world,9d6edb9b-f9d8-4870-bc6f-5d192c28e81a,None,None,None
1,4e10a4f8-06a1-4e4d-8423-cafb7be78371,c3732977-6d76-4192-bccf-38cac682b7b7,FINISHED,,2021-11-16 00:59:20.789000+00:00,2021-11-16 00:59:23.846000+00:00,0.056947,world,4e10a4f8-06a1-4e4d-8423-cafb7be78371,azureuser,/anaconda/envs/azureml_py36/lib/python3.6/site...,LOCAL
2,dd459782-9e85-4544-af88-9eb492ac74ec,c3732977-6d76-4192-bccf-38cac682b7b7,FINISHED,,2021-11-16 01:04:06.568000+00:00,2021-11-16 01:05:02.270000+00:00,0.882664,world,dd459782-9e85-4544-af88-9eb492ac74ec,None,None,None
3,26edd15b-9cce-4a30-b422-c7439aedb06a,c3732977-6d76-4192-bccf-38cac682b7b7,FINISHED,,2021-11-16 01:22:56.810000+00:00,2021-11-16 01:22:58.738000+00:00,0.023711,world,26edd15b-9cce-4a30-b422-c7439aedb06a,azureuser,/anaconda/envs/azureml_py38/lib/python3.8/site...,LOCAL
4,ec8d02d1-dbc3-4fdb-83bd-d24e25db1df7,c3732977-6d76-4192-bccf-38cac682b7b7,FINISHED,,2021-11-16 01:51:58.870000+00:00,2021-11-16 01:52:01.301000+00:00,0.586012,world,ec8d02d1-dbc3-4fdb-83bd-d24e25db1df7,azureuser,/anaconda/envs/azureml_py38/lib/python3.8/site...,LOCAL
5,932e4da8-76af-4f27-89e5-6ce83c994823,c3732977-6d76-4192-bccf-38cac682b7b7,FINISHED,,2021-11-20 00:02:11.545000+00:00,2021-11-20 00:02:14.431000+00:00,0.267684,world,932e4da8-76af-4f27-89e5-6ce83c994823,azureuser,/anaconda/envs/azureml_py38/lib/python3.8/site...,LOCAL
6,09bb00f9-3d7f-4f6e-93b7-856c0c30c68b,c3732977-6d76-4192-bccf-38cac682b7b7,FINISHED,,2021-11-30 21:47:37.287000+00:00,2021-11-30 21:47:39.326000+00:00,0.277389,world,09bb00f9-3d7f-4f6e-93b7-856c0c30c68b,azureuser,/anaconda/envs/azureml_py38/lib/python3.8/site...,LOCAL


In [15]:
# the model folder produced from the run is registered. This includes the MLmodel file, model.pkl and the conda.yaml.
import mlflow
model_path = "model"
model_uri = 'runs:/{}/{}'.format("explore_projects_1637621370_7e433735", model_path) 
mlflow.register_model(model_uri,"mlflow_project")


Registered model 'mlflow_project' already exists. Creating a new version of this model...
2021/11/30 21:49:57 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: mlflow_project, version 2
Created version '2' of model 'mlflow_project'.


<ModelVersion: creation_timestamp=1638308997352, current_stage='None', description='', last_updated_timestamp=1638308997352, name='mlflow_project', run_id='explore_projects_1637621370_7e433735', run_link='', source='azureml://experiments/explore_projects/runs/explore_projects_1637621370_7e433735/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='2'>